# 产品内码与EPC码转换规则
EPC码标头二进制值设定为00010000

In [ ]:
header = '00010000'

## 1、产品内码转EPC码
产品内码分为纯数字型（如:* 49531001649100 * ）和字母数字混合型( 如:* 24310100100148A * )。 
通过1位bit过滤值来区分。0表示数字型，1表示字母数字混合型。

In [3]:
import re

# 预定失败消息
fail = {'valid': False, }

# 异常
class HandleError(RuntimeError):
    def __init__(self, arg):
        self.args = arg

inner_num_code = '49531001649100'
inner_mix_code = '24310100100148A'

def filter(inner_code):
    filter = '0'
    if (re.match('^[\d]+$', inner_code)):
        filter = '0'
    elif (re.match('^(?!\d+$)[\da-zA-Z]+$',inner_code)):
        filter = '1'
    else:
        raise HandleError('产品内码不符合规范')    
    return filter


print('内码为{0}时，过滤值为{1}'.format(inner_num_code,filter(inner_num_code)))
print('内码为{0}时，过滤值为{1}'.format(inner_mix_code,filter(inner_mix_code)))      

内码为49531001649100时，过滤值为0
内码为24310100100148A时，过滤值为1


In [4]:
def padInnerCode(inner_code):
    if (filter(inner_code) == '0'):        
        return '1' + inner_code
    else:
        return inner_code
    
print('内码为{0}时，补位结果为{1}'.format(inner_num_code,padInnerCode(inner_num_code)))
print('内码为{0}时，补位结果为{1}'.format(inner_mix_code,padInnerCode(inner_mix_code)))

内码为49531001649100时，补位结果为149531001649100
内码为24310100100148A时，补位结果为24310100100148A


如果过滤值为0，则在高位补1

In [6]:
def padInnerCode(inner_code):
    if (filter(inner_code) == '0'):        
        return '1' + inner_code
    else:
        return inner_code
    
print('内码为{0}时，补位结果为{1}'.format(inner_num_code,padInnerCode(inner_num_code)))
print('内码为{0}时，补位结果为{1}'.format(inner_mix_code,padInnerCode(inner_mix_code)))

内码为49531001649100时，补位结果为149531001649100
内码为24310100100148A时，补位结果为24310100100148A


如果过滤值为0，直接将十进制字符串转为二进制字符串。如果过滤值为1，将字符串按位编码。
编码规则:
'0'\~'9'转为十进制的0\~9，'a'\~'z'转为十进制的10\~35,'A'\~'Z'转为十进制的36\~61,所以需要每个字符串需要6位即最大值2<sup>6</sup> = 64来表示所有的字符。

In [11]:
def innerCode2Bin(inner_code):
    if (filter(inner_code) == '0'): 
        return bin(int(padInnerCode(inner_code)))
    else:
        for c in padInnerCode(inner_code):
            bin_str += bin(ord(c)-87)
        return bin_str
    
print('内码为{0}时，二进制字符串为{1}'.format(inner_num_code,innerCode2Bin(inner_num_code)))
print('内码为{0}时，二进制字符串为{1}'.format(inner_mix_code,innerCode2Bin(inner_mix_code)))    

内码为49531001649100时，二进制字符串为0b100001111111111101100110001111000111011111001100


UnboundLocalError: local variable 'bin_str' referenced before assignment

### 2、纯数字型内码转EPC步骤
#### 1、在内码高位补1，* 49531001649100 * 转为 * 149531001649100 *

In [13]:
identify = '1'
def handle(event):
    return identify + event

handle(inner_code)

'149531001649100'

### 3、字母数字混合型内码转EPC步骤
#### 1、